In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
html_doc = """
<html><head><title>Web scraping pro datovou analýzu v Pythonu - sedlakovi.org</title></head>
<body>
<p class="title"><b>Web scraping pro datovou analýzu v Pythonu</b></p>

<p class="description">
    Náš kurz Vás naučí jak lze snadno získávat data z webových stránek, 
    které můžete používat k následné analýze anebo reportingu. 
    Příkladem může být analýza kurzu měn, recenzí kaváren nebo cen zboží. 
    Náš kurz Vám ukáže celý postup od identifikace datového zdroje, 
    navržení vhodného scrapingu a přípravy dat pro následnou analýzu. 
</p>

<p class="description">
    Naučíte se:
    <ul>
        <li>Strukturu HTML</li>
        <li>Jak získat obsah webových stránek (knihovna requests)</li>
        <li>Jak vybrat z tohoto obsahu jenom to, co nás zajímá (knihovna BeautifulSoup)</li>
        <li>Jak následně data zpracovat (knihovna pandas)</li>
    </ul>
</p>

<table>
<tr> <th class="table-header"> Naučím se pracovat s: </th>
<td class="table-data">
<a href="http://docs.python-requests.org/en/master/" class="library" id="link1">Requests</a>,
<a href="https://www.crummy.com/software/BeautifulSoup/"  class="library" id="link2">BeautifulSoup</a>,
<a href="https://pandas.pydata.org/" class="library" id="link3">Pandas</a>
</td>
</tr>
</table>
"""

In [3]:
soup = BeautifulSoup(html_doc, 'html.parser')

# Cvičení 3.1

Vyberte odstavec s třídou `title`.

`Web scraping pro datovou analýzu v Pythonu`

In [4]:
soup.find(class_='title').text

'Web scraping pro datovou analýzu v Pythonu'

In [5]:
soup.find(attrs={'class': 'title'}).text

'Web scraping pro datovou analýzu v Pythonu'

In [6]:
soup.find(name='p', attrs={'class': 'title'}).text

'Web scraping pro datovou analýzu v Pythonu'

# Cvičení 3.2

Vyberte všechny položky seznamu

```
 <li>Strukturu HTML</li>
 <li>Jak získat obsah webových stránek (knihovna requests)</li>
 <li>Jak vybrat z tohoto obsahu jenom to, co nás zajímá (knihovna BeautifulSoup)</li>
 <li>Jak následně data zpracovat (knihovna pandas)</li>
```

** Bonus:** zbavte se HTML tagů (např. `Jak získat obsah webových stránek (knihovna requests)`). 

In [7]:
soup.find_all('li')

[<li>Strukturu HTML</li>,
 <li>Jak získat obsah webových stránek (knihovna requests)</li>,
 <li>Jak vybrat z tohoto obsahu jenom to, co nás zajímá (knihovna BeautifulSoup)</li>,
 <li>Jak následně data zpracovat (knihovna pandas)</li>]

In [8]:
[li.text for li in soup.find_all('li')]

['Strukturu HTML',
 'Jak získat obsah webových stránek (knihovna requests)',
 'Jak vybrat z tohoto obsahu jenom to, co nás zajímá (knihovna BeautifulSoup)',
 'Jak následně data zpracovat (knihovna pandas)']

# Cvičení 5.1

Zjistěte na stránkach `https://www.seznam.cz/`, kdo má dnes svátek.

In [9]:
url = 'https://www.seznam.cz/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
soup.find(class_='name-link').text

'Alan'

# Cvičení 5.2

Najděte které filmy se promítaly 8.8.2019 v kině Aero (první v seznamu abecedně seřazených kinech).

`Je mi fajn s.r.o.`

In [10]:
url = 'https://dokina.tiscali.cz/program-kin?date=2019-08-08&cinemasType=nonmulti&tab=cinemas&cinemasOrdering=title'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [11]:
program = soup.select_one('.program')

In [12]:
program.select_one('.cinema-info .name').text

'Aero'

In [13]:
program.select_one('.program-table .name').text

'Je mi fajn s.r.o.'

# Cvičení 5.3

Vytvořte tabulku z [http://quotes.toscrape.com/](http://quotes.toscrape.com/), kde první sloupec bude jméno autora a druhý sloupec bude datum narození. autorů

In [14]:
import pandas as pd

In [15]:
url = 'http://quotes.toscrape.com'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
author_links = soup.select('a[href*="author"]', href=True)
author_links

[<a href="/author/Albert-Einstein">(about)</a>,
 <a href="/author/J-K-Rowling">(about)</a>,
 <a href="/author/Albert-Einstein">(about)</a>,
 <a href="/author/Jane-Austen">(about)</a>,
 <a href="/author/Marilyn-Monroe">(about)</a>,
 <a href="/author/Albert-Einstein">(about)</a>,
 <a href="/author/Andre-Gide">(about)</a>,
 <a href="/author/Thomas-A-Edison">(about)</a>,
 <a href="/author/Eleanor-Roosevelt">(about)</a>,
 <a href="/author/Steve-Martin">(about)</a>]

In [16]:
authors = [['Name', 'Date of birth']]
for author in author_links:
    url = 'http://quotes.toscrape.com{}'.format(author['href'])
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    authors.append([
        soup.select_one('.author-title').text, soup.select_one('.author-born-date').text
    ])
pd.DataFrame(authors[1:], columns=authors[0])    

,Name,Date of birth
0,Albert Einstein,"March 14, 1879"
1,J.K. Rowling,"July 31, 1965"
2,Albert Einstein,"March 14, 1879"
3,Jane Austen,"December 16, 1775"
4,Marilyn Monroe,"June 01, 1926"
5,Albert Einstein,"March 14, 1879"
6,André Gide,"November 22, 1869"
7,Thomas A. Edison,"February 11, 1847"
8,Eleanor Roosevelt,"October 11, 1884"
9,Steve Martin,"August 14, 1945"
